In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math as m
import pandas as pd
from scipy.optimize import curve_fit
from scipy.stats import chisquare
from IPython.display import display, Math

### User Functions

In [35]:
# # Definition of the function to fit # #
def f(x,a,b,c):
    res = a*np.exp(-x/b) + c
    return res

# # Definition of the function to add incertity depending on the scale of the aparata # #
def error(value):
    if value < 1000 and value >= 100:
      sensibility_error = 0.001/m.sqrt(12)
    if value < 100 and value >= 10:
      sensibility_error = 0.0001/m.sqrt(12)
    if value < 10 and value >= 1:
      sensibility_error = 0.00001/m.sqrt(12)
    if value < 1 and value >= 0.1:
      sensibility_error = 0.000001/m.sqrt(12)
    reading_error = 0.0292*value #2.92% of the value
    scale_error = 0.00025*10**5
    err = m.sqrt( (sensibility_error)**2 + (reading_error)**2 +(scale_error)**2 )
    return err

In [36]:
filename = "04_04.xlsx" #Select here the file to take the data
path = "C:/Users/Admin/Desktop/3DJoint/data/"

file = pd.ExcelFile(path+filename)
sheets = file.sheet_names
data = pd.read_excel(path+filename, sheet_name=None) #This puts the data of the day in the variable data

In [37]:
Q = []
errQ = []
T_mean = []
errT = []

In [38]:
%matplotlib qt


for sheet in sheets:
    v = data[sheet]['Voltage']
    v = v[0:-10]
    v_error = [error(val) for val in v]

    t = data[sheet]['Time']
    t = t[0:-10]
    T = data[sheet]['T']
    T_ini = T[0]
    T_fin = T[1]
    T_value = np.mean(T)
    T_error = abs(T_fin-T_ini)/2

    f_0 = data[sheet]['f0'][0]
    errf_0 = (0.003/100)*f_0

    # # Declare units for the data # #
    t_unit = "s"
    v_unit = "mVpp"
    T_unit = "K"
    f_unit = "Hz"

    # # Fit of the data # #
    initial = [300,10,10]
    resval,rescov = curve_fit(f, t, v, initial, sigma = v_error)
    reserr = np.sqrt(np.diag(rescov))
    R2 = 1 - (np.sum( ( v - f( t, *resval) )**2 ) / np.sum( ( v - np.mean(v) )**2 ))

    # # Plot of the data with fit # #

    #sampling time
    h = max([abs((max(t)-min(t))/1000),1])
    fit_time = np.arange(min(t), max(t)+h, h)
    fit_amplitude = resval[0]*np.exp(-fit_time/resval[1])

    fig = plt.figure(figsize=(6,4), dpi=100);
    fig.suptitle(r"Data from {0} of {1}".format(sheet, filename))
    plt.xlabel(r"$t$ ({0})".format(t_unit), size = 10)
    plt.ylabel(r"$Amplitude$ ({0})".format(v_unit), size = 10)
    plt.plot(t,v,'.',c='k', ms=6)
    plt.errorbar(t, v, yerr=v_error, fmt=".k", capsize=3,alpha = 0.65,label="Data")
    plt.plot(fit_time,fit_amplitude,'--',c='red',label="Fit")
    plt.legend()
    #to make it look good
    %config InlineBackend.figure_format='retina'
    plt.tight_layout()
    plt.grid()
    plt.show()

    # # Calculus of the Q-value # #
    Q_value = f_0*resval[1]*m.pi
    Q_error = Q_value*m.sqrt( (reserr[1]/resval[1])**2 + (errf_0/f_0)**2 )
    
    Q.append(Q_value)
    errQ.append(Q_error)
    T_mean.append(T_value)
    errT.append(T_error)
    
    textstr = '\n'.join((
    r'Q = {0:.2f} $\pm$ {1:.2f}'.format(Q_value,Q_error),
    r'T = {0:.2f} $\pm$ {1:.2f} $\mathrm{{{2}}}$'.format(T_value,T_error,T_unit),
    r'$f_0$ = {0} $\pm$ {1} $\mathrm{{{2}}}$'.format(f_0,errf_0,f_unit)))

    props = dict(boxstyle='square', facecolor='white', alpha=1)

    plt.text(0.6, 0.6, textstr, fontsize=10,
            verticalalignment='top',transform=fig.transFigure, bbox=props)

In [39]:
fig = plt.figure(figsize=(6,4), dpi=100);
fig.suptitle('Q vs T')
plt.xlabel(r"$T$ ({0})".format(T_unit), size = 10)
plt.ylabel(r"$Q$",size = 10)
plt.errorbar(T_mean,Q, xerr = errT, yerr = errQ,fmt=".k", capsize=3,label="Data")
plt.legend()
#to make it look good
%config InlineBackend.figure_format='retina'
plt.tight_layout()
plt.grid()
plt.show()